In [1]:
import pandas as pd
import numpy as np

import bbg_symbology as bbs
from bbg_loader_core import *

In [2]:
def merge_dict(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

class ExpiredAliasService:
    def __init__(self, src=None):
        
        self._src = src
            
        if src is not None:
            self.expired = list(pd.read_csv(src, header=None)[1])
        else:
            self.expired = []
            
    def __contains__(self, key):
        return key in self.expired
    
    def __repr__(self):
        return '<{}: {}>'.format(self.__class__, str(self.expired))
    
    def add_expired(self, aliases):
        if isinstance(aliases, list):
            self.expired = self.expired + aliases
        else:
            self.expired.extend([aliases])
        
    def save(self, fname=None):
        if fname is None:
            fname = self._src
            
        if fname is None:
            raise Error('No Filename Specified!')
            
        pd.Series(self.expired).to_csv(fname)

In [3]:
def generate_futures_list(out_path):
    
    # init - flds & default params
    ts_flds =   ['px_last', 'open_int', 'volume']
    meta_flds = ['LAST_TRADEABLE_DT']

    default_params = {'local_path':  out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}
    
    # get all unique roots (alias roots)
    roots = list(FutAliasRef._fut_ref.Alias)
    
    # store outputs
    fut_list = {}
    
    # loop through roots
    for root in roots:
        
        # get all bbg, alias tckrs
        bbg_tckrs  = FutChainRef.get_futures_chain(root)
        alias_tckrs = [FutAliasRef.bbg_to_alias_tckr(tckr) for tckr in bbg_tckrs]
        tckr_list = zip(bbg_tckrs, alias_tckrs)
        
        # loop through ticker combos... and return object
        for bbg, alias in tckr_list:
            if alias not in ExpiredAliases:
                p = merge_dict({'bb_tckr': bbg, 'alias': alias}, default_params)
                fut_list[alias] = BbgSecurity(**p)
                
    return fut_list


def generate_bbg_list(src_file, out_path, ts_flds=['px_last'], meta_flds=['NAME','LONG_COMP_NAME']):
    tckrList = pd.read_csv(src_file)
    default_params = {'local_path': out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}

    secList = {}    
    for (i, (bbg,alias)) in tckrList.iterrows():
        d = merge_dict({'bb_tckr': bbg, 'alias': alias}, default_params)
        secList[bbg] = BbgSecurity(**d)

    return secList


def generate_generic_fut_list(out_path, ts_flds=['px_last'], meta_flds=['NAME','LONG_COMP_NAME']):

    def gen_obj(broot, rmtd, yk, ng, alias_root, alias_suff, dparams):
        bbg_tckr   = broot.upper()+ str(ng) + ' ' + rmtd + ' ' + yk
        alias_tckr = alias_root + str(ng) + '.' + alias_suff

        d = merge_dict({'bb_tckr': bbg_tckr, 'alias': alias_tckr}, dparams)
        return alias_tckr, BbgSecurity(**d)
    
    ts_flds =   ['px_last', 'open_int', 'volume']
    meta_flds = ['NAME', 'PX_POS_MULT_FACTOR', 'EXCH_CODE', 'FUT_EXCH_NAME_LONG', 
                 'CRNCY', 'FUT_TICK_SIZE', 'FUT_GEN_MONTH']

    default_params_ridx = {'local_path': out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}
    default_params_px   = {'local_path': out_path, 'ts_flds': ts_flds[:1], 'meta_flds': meta_flds[:1]}
    
    
    n = 1
    secList = {}    
    for idx, (root, num_gen, yk, alias) in FutAliasRef._fut_ref.iterrows():
        # Return Index
        atckr, bbobj = gen_obj(root, 'B:00_0_D', yk, 1, alias, 'bbd', default_params_ridx)
        secList[atckr] = bbobj
        
        # Price 
        atckr, bbobj = gen_obj(root, 'B:00_0_N', yk, 1, alias, 'bbd.px', default_params_px)
        secList[atckr] = bbobj
        
        # Return Index
        atckr, bbobj = gen_obj(root, 'B:00_0_D', yk, num_gen, alias, 'bbd', default_params_px)
        secList[atckr] = bbobj
        
        # Price
        atckr, bbobj = gen_obj(root, 'B:00_0_N', yk, num_gen, alias, 'bbd.px', default_params_px)
        secList[atckr] = bbobj

    return secList

def update_bbg_securities(bbglist):
    expired = []
    error_list = []
    print('--------------UPDATING {}----------------'.format(db))
    for i, (alias, bbgObj) in enumerate(bbglist.items()):
        print('\n(' + str(i) + ') ' + str(bbgObj))

        try:
            bbgObj.update()
            if bbgObj.is_expired:
                expired.append(bbgObj.alias)
        except:
            try: 
                print('...Error with {}, attempting to load from scratch'.format(bbgObj.bb_tckr))
                bbgObj.load_from_scratch()
                bbgObj.save()
                print('...success!')

                if bbgObj.is_expired:
                    expired.append(bbgObj.alias)
            except:
                print('FATAL ERROR WITH {}'.format(bbgObj.bb_tckr))
                error_list.extend(bbgObj)
    
    return expired, error_list

In [4]:
INPUT_PATH = '../_in/'

FUTURES_MONTHS = {m:i+1 for i,m in enumerate('FGHJKMNQUVXZ')}

FutChainRef = bbs.FuturesChainReference(INPUT_PATH + 'futures_historical_chain.csv')
FutAliasRef = bbs.FuturesAliasService(INPUT_PATH + 'fut_roots.csv')
BbgTckrService = bbs.BloombergTckrService()
ExpiredAliases = ExpiredAliasService('../_in/expired_aliases.csv')


In [5]:
BBG_UPDATE_LIST = {'CoT':           generate_bbg_list(src_file=INPUT_PATH + 'CoT.csv',           out_path='../_bbgDB/CoT/'),
                   'Index':         generate_bbg_list(src_file=INPUT_PATH + 'Index.csv',         out_path='../_bbgDB/Index/'),
                   'FX':            generate_bbg_list(src_file=INPUT_PATH + 'FX.csv',            out_path='../_bbgDB/FX/'), 
                   'InterestRates': generate_bbg_list(src_file=INPUT_PATH + 'InterestRates.csv', out_path='../_bbgDB/InterestRates/'),
                   'Futures_gen':   generate_generic_fut_list(out_path='../_out/Futures_gen/'),
                   'Futures':       generate_futures_list(out_path='../_out/Futures/')}

# print # securities in each
for name, db in BBG_UPDATE_LIST.items():
    print(name, len(db))
    

('Index', 4)
('CoT', 14)
('FX', 68)
('Futures', 20582)
('Futures_gen', 234)
('InterestRates', 46)


In [97]:
databases = list(BBG_UPDATE_LIST.keys())
db = 'InterestRates'
bbglist = BBG_UPDATE_LIST[db]

expired = {}
error_list = {}

for db in databases:
    expired[db], error_list[db] = update_bbg_securities(bbglist)

--------------UPDATING CoT----------------

(0) <BBG SECURITY: tckr=US0001M Index, alias=us.1m, local_file=../_bbgDB/InterestRates/>
...updating security US0001M Index
...loading local file <../_bbgDB/InterestRates/us.1m.pickle> for security=US0001M Index
...updating timeseries for US0001M Index --> loading data from 09/18/2019 to 09/26/2019
...saving US0001M Index to local file <../_bbgDB/InterestRates/us.1m.pickle>

(1) <BBG SECURITY: tckr=GDBR7 Index, alias=eu.7y, local_file=../_bbgDB/InterestRates/>
...updating security GDBR7 Index
...loading local file <../_bbgDB/InterestRates/eu.7y.pickle> for security=GDBR7 Index
...updating timeseries for GDBR7 Index --> loading data from 09/19/2019 to 09/26/2019
...saving GDBR7 Index to local file <../_bbgDB/InterestRates/eu.7y.pickle>

(2) <BBG SECURITY: tckr=CDOR01 Index, alias=cn.1m, local_file=../_bbgDB/InterestRates/>
...updating security CDOR01 Index
...loading local file <../_bbgDB/InterestRates/cn.1m.pickle> for security=CDOR01 Index
.

...saving GUKG10 Index Index to local file <../_bbgDB/InterestRates/uk.10y.pickle>

(24) <BBG SECURITY: tckr=GDBR10 Index, alias=eu.10y, local_file=../_bbgDB/InterestRates/>
...updating security GDBR10 Index
...loading local file <../_bbgDB/InterestRates/eu.10y.pickle> for security=GDBR10 Index
...updating timeseries for GDBR10 Index --> loading data from 09/19/2019 to 09/26/2019
...saving GDBR10 Index to local file <../_bbgDB/InterestRates/eu.10y.pickle>

(25) <BBG SECURITY: tckr=BP0001M Index, alias=uk.1m, local_file=../_bbgDB/InterestRates/>
...updating security BP0001M Index
...loading local file <../_bbgDB/InterestRates/uk.1m.pickle> for security=BP0001M Index
...updating timeseries for BP0001M Index --> loading data from 09/18/2019 to 09/26/2019
...saving BP0001M Index to local file <../_bbgDB/InterestRates/uk.1m.pickle>

(26) <BBG SECURITY: tckr=RBATCTR Index, alias=au.cb, local_file=../_bbgDB/InterestRates/>
...updating security RBATCTR Index
...loading local file <../_bbgDB/In

...saving US0001M Index to local file <../_bbgDB/InterestRates/us.1m.pickle>

(1) <BBG SECURITY: tckr=GDBR7 Index, alias=eu.7y, local_file=../_bbgDB/InterestRates/>
...updating security GDBR7 Index
...loading local file <../_bbgDB/InterestRates/eu.7y.pickle> for security=GDBR7 Index
...updating timeseries for GDBR7 Index --> loading data from 09/19/2019 to 09/26/2019
...saving GDBR7 Index to local file <../_bbgDB/InterestRates/eu.7y.pickle>

(2) <BBG SECURITY: tckr=CDOR01 Index, alias=cn.1m, local_file=../_bbgDB/InterestRates/>
...updating security CDOR01 Index
...loading local file <../_bbgDB/InterestRates/cn.1m.pickle> for security=CDOR01 Index
...updating timeseries for CDOR01 Index --> loading data from 09/19/2019 to 09/26/2019
...saving CDOR01 Index to local file <../_bbgDB/InterestRates/cn.1m.pickle>

(3) <BBG SECURITY: tckr=GSGB7YR Index, alias=se.7y, local_file=../_bbgDB/InterestRates/>
...updating security GSGB7YR Index
...loading local file <../_bbgDB/InterestRates/se.7y.pick

...saving GDBR10 Index to local file <../_bbgDB/InterestRates/eu.10y.pickle>

(25) <BBG SECURITY: tckr=BP0001M Index, alias=uk.1m, local_file=../_bbgDB/InterestRates/>
...updating security BP0001M Index
...loading local file <../_bbgDB/InterestRates/uk.1m.pickle> for security=BP0001M Index
...updating timeseries for BP0001M Index --> loading data from 09/18/2019 to 09/26/2019
...saving BP0001M Index to local file <../_bbgDB/InterestRates/uk.1m.pickle>

(26) <BBG SECURITY: tckr=RBATCTR Index, alias=au.cb, local_file=../_bbgDB/InterestRates/>
...updating security RBATCTR Index
...loading local file <../_bbgDB/InterestRates/au.cb.pickle> for security=RBATCTR Index
...updating timeseries for RBATCTR Index --> loading data from 05/07/2019 to 09/26/2019
...saving RBATCTR Index to local file <../_bbgDB/InterestRates/au.cb.pickle>

(27) <BBG SECURITY: tckr=GNOR2YR Index, alias=no.2y, local_file=../_bbgDB/InterestRates/>
...updating security GNOR2YR Index
...loading local file <../_bbgDB/Intere

In [99]:
print(error_list)

{'Index': [], 'CoT': []}


--------------UPDATING InterestRates----------------

(0) <BBG SECURITY: tckr=US0001M Index, alias=us.1m, local_file=../_bbgDB/InterestRates/>
...updating security US0001M Index
...loading local file <../_bbgDB/InterestRates/us.1m.pickle> for security=US0001M Index
...updating timeseries for US0001M Index --> loading data from 09/17/2019 to 09/26/2019
...saving US0001M Index to local file <../_bbgDB/InterestRates/us.1m.pickle>

(1) <BBG SECURITY: tckr=GDBR7 Index, alias=eu.7y, local_file=../_bbgDB/InterestRates/>
...updating security GDBR7 Index
...loading local file <../_bbgDB/InterestRates/eu.7y.pickle> for security=GDBR7 Index
...updating timeseries for GDBR7 Index --> loading data from 09/18/2019 to 09/26/2019
...saving GDBR7 Index to local file <../_bbgDB/InterestRates/eu.7y.pickle>

(2) <BBG SECURITY: tckr=CDOR01 Index, alias=cn.1m, local_file=../_bbgDB/InterestRates/>
...updating security CDOR01 Index
...loading local file <../_bbgDB/InterestRates/cn.1m.pickle> for security=CDOR

...saving USGG7YR Index to local file <../_bbgDB/InterestRates/us.7y.pickle>

(23) <BBG SECURITY: tckr=GUKG10 Index Index, alias=uk.10y, local_file=../_bbgDB/InterestRates/>
...updating security GUKG10 Index Index
...loading local file <../_bbgDB/InterestRates/uk.10y.pickle> for security=GUKG10 Index Index
...updating timeseries for GUKG10 Index Index --> loading data from 09/18/2019 to 09/26/2019
...saving GUKG10 Index Index to local file <../_bbgDB/InterestRates/uk.10y.pickle>

(24) <BBG SECURITY: tckr=GDBR10 Index, alias=eu.10y, local_file=../_bbgDB/InterestRates/>
...updating security GDBR10 Index
...loading local file <../_bbgDB/InterestRates/eu.10y.pickle> for security=GDBR10 Index
...updating timeseries for GDBR10 Index --> loading data from 09/18/2019 to 09/26/2019
...saving GDBR10 Index to local file <../_bbgDB/InterestRates/eu.10y.pickle>

(25) <BBG SECURITY: tckr=BP0001M Index, alias=uk.1m, local_file=../_bbgDB/InterestRates/>
...updating security BP0001M Index
...loading lo

...saving CABROVER Index to local file <../_bbgDB/InterestRates/cn.cb.pickle>

(44) <BBG SECURITY: tckr=SZLTHIGH Index, alias=ch.cb, local_file=../_bbgDB/InterestRates/>
...updating security SZLTHIGH Index
...loading local file <../_bbgDB/InterestRates/ch.cb.pickle> for security=SZLTHIGH Index
...updating timeseries for SZLTHIGH Index --> loading data from 06/21/2018 to 09/26/2019
...saving SZLTHIGH Index to local file <../_bbgDB/InterestRates/ch.cb.pickle>

(45) <BBG SECURITY: tckr=GNZGB2 Index, alias=nz.2y, local_file=../_bbgDB/InterestRates/>
...updating security GNZGB2 Index
...loading local file <../_bbgDB/InterestRates/nz.2y.pickle> for security=GNZGB2 Index
...updating timeseries for GNZGB2 Index --> loading data from 09/18/2019 to 09/26/2019
...saving GNZGB2 Index to local file <../_bbgDB/InterestRates/nz.2y.pickle>


In [93]:
error_list

[]

In [38]:
ExpiredAliases.save()

<__main__.ExpiredAliasService: ['cl.Z05', 'cl.Z06', 'cl.Z09']>